In [1]:
import seaborn as sns
import pandas as pd 
import numpy as np
from scipy.spatial.distance import squareform, pdist
import matplotlib.pyplot as plt
import torch
import anndata as an
import scanpy as sc
import umap
import gc

from datasets import Dataset, load_from_disk
from datasets import load_dataset
from geneformer import EmbExtractor

sns.set_style('white')

/home/cstansbu/miniconda3/envs/geneformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model = "/nfs/turbo/umms-indikar/shared/projects/HSC/data/geneformer/fine_tuned_models/geneformer-6L-30M_CellClassifier_cardiomyopathies_220224"
model = "/nfs/turbo/umms-indikar/shared/projects/HSC/data/geneformer/geneformer-12L-30M/"
data_path = "/scratch/indikar_root/indikar1/cstansbu/HSC/geneformer_inputs/iHSC.dataset"
outpath = "/scratch/indikar_root/indikar1/cstansbu/geneformer"

In [3]:
def load_and_subset_data(data_path: str, num_cells: int = 100) -> pd.DataFrame:
    """Loads a dataset from disk, selects a subset of cells, and converts it to a Pandas DataFrame.

    Args:
        data_path (str): Path to the dataset file.
        num_cells (int, optional): Number of cells to include in the subset (default: 100).

    Returns:
        pd.DataFrame: The subset of data as a Pandas DataFrame.
    """

    data = load_from_disk(data_path)
    if num_cells > len(data):
        raise ValueError(f"Requested subset size ({num_cells}) exceeds dataset length ({len(data)})")

    data_subset = data.select([i for i in range(num_cells)])
    df = data_subset.to_pandas()

    return df

# Example usage:
df = load_and_subset_data(data_path, num_cells=50)
df.head()

,input_ids,cell_type,n_counts,dataset,length
0,"[12845, 8704, 947, 18004, 8351, 8431, 2896, 88...",iHSC,5784.0,iHSC,2048
1,"[12845, 8351, 1610, 24483, 3001, 2896, 8836, 9...",iHSC,4842.0,iHSC,2048
2,"[8704, 8351, 18004, 8431, 947, 16181, 24483, 1...",iHSC,3812.0,iHSC,1511
3,"[12845, 8351, 18004, 8704, 947, 3001, 8836, 16...",iHSC,3039.0,iHSC,1458
4,"[12845, 8351, 8431, 8704, 1912, 947, 3011, 198...",iHSC,12733.0,iHSC,2048


In [4]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [ ]:
torch.cuda.empty_cache()

n_cells = 10000

# initiate EmbExtractor
embex = EmbExtractor(model_type="CellClassifier",
                     num_classes=1,
                     max_ncells=n_cells,
                     emb_layer=-1,
                     emb_label=["cell_type", "dataset"],
                     labels_to_plot=["cell_type", "dataset"],
                     forward_batch_size=10,
                     nproc=16)

# extracts embedding from input data
# input data is tokenized rank value encodings generated by Geneformer tokenizer (see tokenizing_scRNAseq_data.ipynb)
# example dataset: https://huggingface.co/datasets/ctheodoris/Genecorpus-30M/tree/main/example_input_files/cell_classification/disease_classification/human_dcm_hcm_nf.dataset
embs = embex.extract_embs(model,
                          data_path,
                          outpath,
                          "test")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /nfs/turbo/umms-indikar/shared/projects/HSC/data/geneformer/geneformer-12L-30M/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 72%|███████▏  | 715/1000 [14:13<05:23,  1.13s/it]

In [6]:
# break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
def embedding_to_adata(df):
    """Converts a Pandas DataFrame with an embedding to an AnnData object.
    
    Args:
        df (pd.DataFrame): The input DataFrame, with numerical embedding columns and
            optional metadata columns.

    Returns:
        an.AnnData: The converted AnnData object.
    """
    
    cell_index = [f"C{x}" for x in range(df.shape[0])]
    
    metadata_cols = [col for col in df.columns if not str(col).isdigit()]
    obs = df[metadata_cols].copy()
    obs.index = cell_index
    
    df = df.drop(columns=metadata_cols)
    var_index = [f"D{x}" for x in range(df.shape[1])]
    
    adata = an.AnnData(df.to_numpy())
    adata.obs_names = cell_index
    adata.var_names = var_index
    adata.obs = obs

    return adata

    
adata = embedding_to_adata(embs)
adata

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(adata.X[:, 100:200])

adata.obs['UMAP 1'] = embedding[:, 0]
adata.obs['UMAP 2'] = embedding[:, 1]

adata

In [ ]:
sns.scatterplot(data=adata.obs,
                x='UMAP 1',
                y='UMAP 2',
                hue='dataset',
                s=10)

sns.move_legend(plt.gca(), 
                loc='upper right',
                bbox_to_anchor=(1.7, 1))

plt.gca().set_aspect('equal')

In [ ]:
break

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_neighbors=21)
sc.tl.umap(adata, min_dist=0.2)

adata

In [ ]:
sc.pl.umap(adata, 
           color=['dataset'])

In [ ]:
break

In [ ]:
df = embs.copy()

sns.scatterplot(data=df,  
                x=0,
                y=2,
                alpha=0.6,
                s=10,
                palette="Set1",
                # legend=False,
                hue='dataset')

sns.move_legend(plt.gca(), 
                loc='upper right',
                bbox_to_anchor=(1.4, 1))
